<h4>Lily Grumbach
    <br>M1 Humanités numériques - Université PSL</h4>
<h1><center>Rendu TAIS-TAL </center></h1>
<h2><center>Partie TAL</center></h2>
<h3><center>2/3 : Reconnaissance d'entités nommées sur Doccano  -   Evaluation</center></h3>

<b><u>PLAN:</u></b>
1) Exploration des données
<br>
**2) NER avec Doccano**

    a) initialisation des données
    b) Exploration de la NER préalable de Spacy
    c) Mise en place de la stratégie pour l'annotation sur Doccano
    d) Entraînement par 5-fold cross validation
    e) Evaluation des annotations

3) Application du NER au df

**Enjeux de la partie 2:**
Les deux revues scientifiques étudiées reposent sur trois éléments consitutifs: 
- **Thématiques** : 
    - Hygiène 
    - Pathologies dites "exotiques" ou "coloniales"
    - Liens avec les sciences humaines et sociales 
    - Relations avec les découvertes récentes
- **Terrains spécifiques**  : 
    - Colonies et protectorats français
    - Batiments navals de la marine française
    - Autres territoires hors métropole et possessions françaises
    - Dans certains cas des laboratoires
- **Corps de métier** Population particulière de rédacteurs que sont les membres du corps de santé des colonies et du corps de médecine navale(fait l'objet d'une analyse de réseau non prise en considération dans ce devoir)


**Objectifs de la partie 2:**
- Entraîner une NER sur notre BDD

<br></br>
**L'objectif à long terme** de cet entraînement NER est de :
- Mettre en place une NER qui pourrait être appliquée au full text de chacune des revues une fois les OCR nettoyées. 
- Appliquer cette NER à la revue des Archives de médecine navale et coloniale (1892-1898)
- Alimenter une future base de données relationnelle


**Limites de mon travail:** Il me sera nécessaire de mesurer si elle fonctionne bien à la fois sur le reste de la période étudiée ainsi que sur le Bulletin de la Société de Pathologie exotique



In [1]:
#Généraux : 
import spacy
import fr_core_news_md
import pandas as pd
import numpy as np
import re


#Importation de mon module d'éval
import evaluation_NER
from evaluation_NER import Matrics

#Importation de mon module d'entrainement
import Training_Doccano
from Training_Doccano import pandas_to_doccano
from Training_Doccano import train_data
from Training_Doccano import train_nlp

#Autres
import pickle
from collections import Counter
from os import path
from pathlib import Path
import glob


ModuleNotFoundError: No module named 'fr_core_news_md'

In [2]:
## Exemple donné dans le tutoriel :

if __name__ == "__main__":
    
    sents_true_labels = [[{'start_idx': 0, 'end_idx': 1, 'text': 'Foreign Ministry', 'label': 'ORG'}, 
                          {'start_idx': 3, 'end_idx': 4, 'text': 'Shen Guofang', 'label': 'PER'},
                          {'start_idx': 6, 'end_idx': 6, 'text': 'Reuters', 'label': 'ORG'}]]
                        
    sents_pred_labels = [[{'start_idx': 3, 'end_idx': 3, 'text': 'Shen', 'label': 'PER'},
                          {'start_idx': 6, 'end_idx': 6, 'text': 'Reuters', 'label': 'ORG'}]]

matrics = Matrics(sents_true_labels, sents_pred_labels)
matrics.cal_confusion_matrices()
matrics.print_confusion_matrices()
matrics.cal_scores()
matrics.print_scores() 

ORG: {'TP': 1, 'TN': 0, 'FP': 0, 'FN': 1}
PER: {'TP': 0, 'TN': 0, 'FP': 1, 'FN': 1}
ORG: f1 0.6667, precision 1.0000, recall 0.5000
PER: f1 0.0000, precision 0.0000, recall 0.0000


## Tout notre enjeu est d'obtenir de notre texte les listes de listes de dictionnaires et faire correspondre les gold entities avec les prédictions. La tache n'est pas simple puisque nous devons itérer pour chacun de nos titres.

### Application du nouveau modèle nlp sur le test_set1:

In [3]:
#####PREDICTION TEST_SET1

#importation du fichier txt de test_set1 :
with open ("./Notebook_data/Training_Doccano/train-test_sets_articles/test_set1") as f:
    test_set1_list = f.readlines()
# test_set1 = "".join(test_set1_list)
# print(test_set1_list)

#Application du nouveau nlp au test set
nlp_path = "./Notebook_data/Training_Doccano/trained_nlp/nlp1"
nlp_test1 = spacy.load(nlp_path)

#Modèle prédictif sur le test_set1
Liste_pred = []
for article in test_set1_list:
    article = re.sub(r'(^ {1,2})',"",article)
    pred = nlp_test1(article)
    ents = list(pred)
    Liste_pred.append(pred)

In [4]:
#Vérification que le nlp est bien passé:
Liste_pred[5].ents

(géographie médicale, fièvre bilieuse hémoglobinurique)

<b>Commentaire : </b>
Le NLPsemble être assez correct dansl'ensemble!On remarque toutefois des incohérences : 
- "tsé" comme SHS
- "Terre", "malgaches" comme GPE
- "Dr Barbe" comme ORG

### 3.1.3. Evaluation du nouveau NLP

Je dois pour cela annoter moi-même le test_set1 come "gold standard" et en comparer les résultats avec ceux de la prédiction juste au dessus

In [5]:
#Après annotation manuelle, j'importe ke test_set1-annotated 
path_testset1annotated="./DOCCANO/doccano-output/test_set1-annotated.jsonl"

#Conversion au format TRAIN_DATA pour la manipulation
gold_testset1=train_data(path_testset1annotated)

In [6]:
# gold_testset1

Création d'un dataframe de nos articles que l'on va remplir avec les annotations pour y vois plus clair :

In [7]:
df_gold = pd.DataFrame(gold_testset1)
df_gold = df_gold.rename(columns={"entities":"gold_annotations","text":"article_titre"})
# df_gold

Etrange mais pour avoir les données dansl'exacte même disposition, on va en fait ce qu'on va faire pour avoir les exactes mêmes types de données en gold et en prediction c'est qu'on va traiter les prédictions comme si on allait les exporter puis importer de Doccano

In [8]:
df=pd.DataFrame(df_gold["article_titre"])

In [9]:
pandas_to_doccano(df,nlp_test1,"./Notebook_data/Apply_Doccano-df/prediction_testset1.csv")            

pred_data = train_data("./Notebook_data/Apply_Doccano-df/prediction_testset1.csv")

File created!


In [10]:
df_pred=pd.DataFrame(pred_data)
df_pred=df_pred.rename(columns={"entities":"gold_annotations","text":"article_titre"})

df_pred_gold = pd.merge(df_gold,df_pred,how="left",on="article_titre")
df_pred_gold= df_pred_gold.rename(columns={"gold_annotations_x":"gold_annotations","gold_annotations_y" : "pred_annotations"})

In [11]:
# df_pred_gold

<b>Commentaire manuel : </b>

In [15]:
Sample1 = pd.read_pickle('./Notebook_data/Apply_Doccano-df/Sample1_df_pred_gold.pkl')
Sample2 = pd.read_pickle('./Notebook_data/Apply_Doccano-df/Sample2_df_pred_gold.pkl')
Sample2

,article_titre,gold_annotations,pred_annotations
192,Variétés. / Concours sur l'organisation des se...,[],[]
19,Géographie médicale. Notes sur le Yun-Nan (Chi...,"[(34, 41, GPE), (43, 48, GPE), (0, 19, SHS)]","[(0, 19, SHS), (34, 41, GPE), (43, 48, GPE)]"
22,Variétés. Préjugés des Hindous sur la variole.,"[(23, 30, SHS), (38, 45, PATH)]","[(23, 30, SHS), (38, 45, PATH)]"
54,De l'emploi du paddy (riz non décortiqué) dans...,"[(50, 58, PATH)]","[(15, 20, CHEM-LABO), (50, 58, PATH)]"
86,Analyse des eaux servant actuellement à l'alim...,"[(70, 77, GPE)]",[]
288,Vingt hystérectomies abdominales. Résultats te...,[],[]
3,Épidémie d'ophtalmie purulente observée dans l...,"[(11, 30, PATH), (49, 82, GPE), (139, 149, ZOO...","[(11, 30, PATH)]"
93,Statistique générale de la morbidité et de la ...,[],[]
220,"Le choléra à bord de la « Comète », par le Dr ...","[(3, 10, PATH)]","[(3, 10, PATH)]"
113,Maladie du sommeil. Trypanosomiases animales e...,"[(64, 80, GPE), (48, 55, ZOO-ENTO), (20, 44, Z...","[(0, 18, PATH), (64, 80, GPE)]"


In [17]:
Sample2.iloc[9,1]

[(64, 80, 'GPE'), (48, 55, 'ZOO-ENTO'), (20, 44, 'ZOO-ENTO'), (0, 18, 'PATH')]

## Calcul manuel 
#### Sample 1

::: center
  ------- ------- ------- -------
                           Total
           TP=9    FP=2    r1=11
           FN=4    TN=80   r2=19
   Total   13=c1   17=c2   30=t
  ------- ------- ------- -------
:::

$$AccuracySample1 =  \frac{TP + TN}{TP + TN + FP + FN} = \frac{9+80}{9+80+4+2} = 0.93$$
$$PrecisionSample1 =  \frac{TP}{TP + FP} = \frac{9}{9+4} = 0.69$$



#### Sample 2
::: center
  ------- ------- ------- -------
                           Total
           TP=10   FP=1    r1=11
           FN=5    TN=80   r2=85
   Total   15=c1   81=c2   96=t
  ------- ------- ------- -------
:::

$$AccuracySample2 =  \frac{TP + TN}{TP + TN + FP + FN} = \frac{10+80}{10+80+1+5} = 0.94$$
$$PrecisionSample1 =  \frac{TP}{TP + FP} = \frac{10}{10+5} = 0.66$$


### On observe que notre NLP a une assez bonne accuracy mais que la précision est par contre mauvaise.


## Pour rappel, on cherche à avoir un modèle du type : 

`sents_true_labels = [[{'start_idx': pos0, 'end_idx': pos1, 'text': 'Token', 'type': 'token_label'}, .....]]`
   
   
   ==> Donc dans mon cas je dois avoir pour CHAQUE ARTICLE,une liste contenant les dictionnaires associés 
   
`sents_pred_labels = [[{'start_idx': pos0, 'end_idx': pos1, 'text': 'Token', 'type': 'token_label'}, .....]]`


Donc il faut que pour chaque annotation, je passe la liste de tuples en liste de dictionnaires ! 

In [ ]:
entity_set = ['SHS', 'GPE', 'PATH', 'ZOO-ENTO', 'LOC', 'CHEM-LABO', 'ORG']

In [ ]:
df_pred_gold = df_pred_gold.assign(gold_sents=lambda df_pred_gold: None)
df_pred_gold = df_pred_gold.assign(pred_sents=lambda df_pred_gold: None)


for i in range (len(df_pred_gold)):
    sents_true_labels=[]
    dico_tok={'start_idx': None, 'end_idx': None, 'text':None, 'label': None}
    gold_annotation=df_pred_gold.iloc[i,1] 
    pred_annotation =df_pred_gold.iloc[i,2]
    text=df_pred_gold.iloc[i,0]
    
    for token in gold_annotation:
        if token[2] in entity_set:
            dico_tok['start_idx']=token[0]
            dico_tok['end_idx']=token[1]
            dico_tok['label']=token[2]
            dico_tok['text']=text[token[0]:token[1]]
    sents_true_labels.append(dico_tok)
    df_pred_gold.iloc[i,3]=sents_true_labels
    
    dico_tok2={'start_idx': None, 'end_idx': None, 'text':None, 'label': None}
    for token in pred_annotation:
        if token[2] in entity_set:
            dico_tok2['start_idx']=token[0]
            dico_tok2['end_idx']=token[1]
            dico_tok2['label']=token[2]
            dico_tok2['text']=text[token[0]:token[1]]
    sents_pred_labels.append(dico_tok2)
    df_pred_gold.iloc[i,4]=sents_pred_labels

Je ne comprends pas pourquoi cela ne fonctionne pas dans le cas de pred_annotations...

In [ ]:
df_pred_gold

In [ ]:
sents_true_labels=df_pred_gold["gold_sents"].tolist()
sents_pred_labels=df_pred_gold["pred_sents"].tolist()

In [ ]:
matrics = Matrics(sents_true_labels, sents_pred_labels)
matrics.cal_confusion_matrices()
matrics.print_confusion_matrices()
matrics.cal_scores()
print("\n")
matrics.print_scores() 

Pour avoir une idée du nombre de d'annotations prédites/dans le gold.

In [ ]:
print('pred : ',len(Liste_pred),'\ngold : ',len(gold_testset1))

##Effacer la dernière entrée de Liste_pred qui est en fait vide:
# Liste_pred = Liste_pred[:-1]
print("\n\nlast element deleted\n\n")
print('pred : ',len(Liste_pred),'\ngold : ',len(gold_testset1))

Chaque entité nommée a un : un id et un label dédié au fil des annotations. Ici on cherche à créer un dictionnaire de type {id_EN : label_EN}

In [ ]:
types_labels = {0:0} #On garde cette première valeur pour les tokensqui ne sont pas des EN
for pred in Liste_pred:
    for ent in pred.ents: 
        types_labels[ent.label] = ent.label_
types_labels

On créé plusieurs listes avec les tokens des EN <u>prédites</u> : 

In [ ]:
pred_tokens = []
pred_entities = []
pred_positions = []

for i in range (len(Liste_pred)):
    for token in Liste_pred[i]:
        pred_tokens.append(token)
        pred_entities.append(types_labels[token.ent_type]) 
        pred_positions.append((i,token, token.idx,token.idx + len(token),types_labels[token.ent_type])) #Position de chacun des token

on obtient une liste `pred_position` composée de tupples contenant:

    1) le numéro de l'article 
    2) le token
    3) la pos1 du token
    4) la pos2 du token
    5) le label

In [ ]:
# pred_positions

In [ ]:
for i in range(1,5247):
    list_token_article=[]
    token = pred_positions[i]
    nexttoken=pred_positions[i-1]
    while token[0]==nexttoken[0]:
        if token[4] != 0:
            newtuple=(token[2],token[3],token[4])
            list_token_article.append(newtuple)
print(list_token_article)

on obtient une liste `pred_entities` composée de l'ensemble des labels d'entité nommée associé

L'enjeu est alors de comparer ces entités à notre `gold_testset1`. 

Ici, on découpe le `gold_testset1` en fonction des tokens de pred `pred_position` 

In [ ]:
gold_tokens = []  

i = 0
pos = pred_positions[i]

for pos in pred_positions:
    i = pos[0] #le numéro de l'article
    gold_art = gold_testset1[i] #le dictionnaire de l'article
    gold_tokens.append(gold_art["text"][pos[2]:pos[3]])#découpage de gold enfonction de pred

In [ ]:
for i in pred_positions:
    i = pos[0] #le numéro de l'article
    gold_art = gold_testset1[i] #le dictionnaire de l'article
    gold_tokens.append(gold_art["text"][pos[2]:pos[3]])#découpage de gold enfonction de pred

In [ ]:
# gold_tokens

Maintenant, on créé un dictionnaire qui aura pour clé les labels d'EN et qui au final accueillera les token et positions associés

In [ ]:
entity_set = ['SHS', 'GPE', 'PATH', 'ZOO-ENTO', 'LOC', 'CHEM-LABO', 'ORG']

list_ents={}
for i in entity_set:
    list_ents[i] = None

In [ ]:
# slice the gold standard text using the token positions from the prediction
gold_tokens = []  

i = 0
pos = pred_positions[i]

for pos in pred_positions:
    i = pos[0]
    if pos[0] == i:
        gold_art = gold_testset1[i]
        gold_tokens.append(gold_art["text"][pos[2]:pos[3]])
# gold_tokens

Pour rappel, on cherche à avoir un modèle du type : 

`sents_true_labels = [[{'start_idx': pos0, 'end_idx': pos1, 'text': 'Token', 'type': 'token_label'}, .....]]`
   
   
   ==> Donc dans mon cas je dois avoir pour CHAQUE ARTICLE,une liste contenant les dictionnaires associés 
   
`sents_pred_labels = [[{'start_idx': pos0, 'end_idx': pos1, 'text': 'Token', 'type': 'token_label'}, .....]]`

In [ ]:
list_art_ents = {}
entpos = []
gold_entity_ranges = {}
for i in range (len(gold_testset1)): #pour chaque article
    for entity in gold_testset1[i]['entities']: #pour chaque token annotés dans gold
        liste_pos = list(range(entity[0], entity[1]))
        list_art_ents[str(i)]=liste_pos
        for ent in entity_set:#Pour chaque label
            if entity[2] == ent:
                entpos += (list(range(entity[0], entity[1])))
            gold_entity_ranges[ent] = entpos

In [ ]:
gold_entity_ranges.keys()

In [ ]:
gold_entities = []

for pos in pred_positions: #Pour l'ensemble des POS
    is_entity = False #set pardéfaut
    for label in entity_set: #pour chaque label
#         print(label)
        if set(range(pos[2], pos[3])) & set(gold_entity_ranges[label]): #Si on trouve le set de lettres dans le dico du label en question 
            is_entity = True #le pos est bien un NER
            gold_entities.append(label) #on ajoute ce label à la liste 
            break
    if not is_entity:
        gold_entities.append(0) #on met un 0 si le pos n'est pas un NER de Gold


In [ ]:
results = {}

for label in entity_set:
    label_gold = [1 if ent==label else 0 for ent in gold_entities]
    label_pred = [1 if ent==label else 0 for ent in pred_entities]

    results[label] = (label_gold, label_pred)

In [ ]:
df_labels=pd.DataFrame(results,index=["gold","pred"])

In [ ]:
df_labels

Autres essais : 

In [ ]:
dico_gold_ents={'start_idx': None, 'end_idx': None, 'text': None, 'label': None}

def gold_to_list():
    listetoto=[]
    df2=pd.DataFrame()
    for i in range(len(gold_testset1)):
        Liste_total=[]
        Liste_dicos_par_article=[]
        for ent in gold_testset1[i]['entities']:
            dico_gold_ents["start_idx"] = ent[0]
            dico_gold_ents["end_idx"] = ent[1]
            dico_gold_ents["label"] = ent[2]
            dico_gold_ents["text"] = gold_testset1[i]['text'][ent[0]:ent[1]]
            Liste_dicos_par_article.append(dico_gold_ents)
        Liste_total.append(Liste_dicos_par_article)
        df=pd.DataFrame(Liste_dicos_par_article)
        df2=pd.concat([df2,df])
        return(df2)
